# Yolo V4 Discovery
This notebook is to play with the sample code on Colab

Right now this:

    * generates the weights on the nhd-chicken google drive under My Drive/yolov4

## Warning

This notebook relies on weights that are stored on nhdchicken's google drive and therefore requires its credential to mount it.

In [2]:
%%bash
if [  -d "/content/nhd-colab" ]
then
   echo "Environment already initialized"
elif [ -d "/content" ] && [ ! -d "/content/nhd-colab" ]
then
    cd /content || exit 1;
    echo "Installing https://github.com/nhdchicken/nhd-colab.git"
    git clone --recurse-submodules https://github.com/nhdchicken/nhd-colab.git || exit 1;
    cd nhd-colab || exit 1;
else
    echo "Not running in Colab - going to root of repos"
    cd `git rev-parse --show-toplevel` || exit 1;
fi
pwd
pip install utils/nhdcolab > /dev/null 2>&1 || exit 1;
nhdcolab > /dev/null 2>&1 || exit 1;
echo "Great Success!"

Not running in Colab - going to root of repos
/Users/lpbrac/gitlab/pyops/nhd/nhd-colab
Great Success!


In [3]:
# This line instal the yolov4-tf components 
! nhdcolab init yolov4-tf


NHD_COLAB_REPOS_ROOT=/Users/lpbrac/gitlab/pyops/nhd/nhd-colab OK!
Google Drive not mounted
going to repos root dir /Users/lpbrac/gitlab/pyops/nhd/nhd-colab
loading install config /Users/lpbrac/gitlab/pyops/nhd/nhd-colab/install.yml
installing components
installing component yolov4-tf
[running]pip3 install install -r yolov4/tensorflow/requirements.txt OK!


In [ ]:
from nhdcolab.environment import NHDEnvironment
NHD_ENV = NHDEnvironment(gdrive_mount=True)

This cell:
    connects to the google drive
    converts weights to TF Lite with different quantization
    
    It set the following global variables:
    
    * YOLOV4_SUB_MOD_ROOT: The root of the YOLOV4 sub module
    * YOLOV4_WEIGHTS_DIR: Root directory where all weight are to be stored
    * YOLOV4_COCO_TFLITE_WEIGHTS: TF Lite weight
    * YOLOV4_COCO_TFLITE_FP16_WEIGHTS: 16 bits quantization
    * YOLOV4_COCO_TFLITE_UINT8_WEIGHTS: Unsigned 8 bit fixed point (UINT9)
    
Note that YOLOV4_COCO_TFLITE_UINT8_WEIGHTS fails in Tensor Flow right now
    
**ATTENTION**
Follow the instructions to connect the nhd-chicken google drive

In [ ]:
""" When running on Colab, this cell attempt to mount the Google drive 
    to access the weights located at /content/drive/My Drive/yolov4

    If not in Colab, those weight are expected to be found under 
    nhd-colab/yolov4/tensorflow/data

    When not running in Colab, you can download the weight from 
    the Goodle Drive and store ir under the nhd-colab/yolov4/tensorflow/data
    directory or set the content in any directory and set the 
    YOLOV4_WEIGHTS_DIR environment variable to that location.
"""
import os
import warnings
import pathlib
import subprocess

YOLOV4_SUB_MOD_ROOT = NHD_ENV.NHD_COLAB_REPOS_ROOT / 'yolov4' / 'tensorflow'
YOLOV4_CONVERT_TFLITE_SCRIPT = 'convert_tflite.py '
YOLOV4_WEIGHTS_DIR = NHD_ENV.NHD_COLAB_DRIVE / 'models'/ 'yolov4'
COCO_DATA_SET_VAL = YOLOV4_SUB_MOD_ROOT / 'data' / 'dataset' / 'val2017.txt'

YOLOV4_COCO_TFLITE_WEIGHTS = YOLOV4_WEIGHTS_DIR / 'yolov4.tflite'
YOLOV4_COCO_TFLITE_FP16_WEIGHTS = YOLOV4_WEIGHTS_DIR / 'yolov4-fp16.tflite'
YOLOV4_COCO_TFLITE_UINT8_WEIGHTS = YOLOV4_WEIGHTS_DIR / 'yolov4-int8.tflite'

def convert(output, quant_mode=""):
    cmd = f"python3 {YOLOV4_CONVERT_TFLITE_SCRIPT} --weights '{YOLOV4_COCO_WEIGHTS}' --output '{output}' {quant_mode} --dataset {COCO_DATA_SET_VAL}"
    print(f"converting {YOLOV4_COCO_WEIGHTS} to  {output} ")
    print(f">> running command {cmd}")
    result = subprocess.run(cmd, stderr=subprocess.PIPE, shell=True)
    if result.returncode:
        raise OSError(f"command {cmd} failed\b {result.stderr.decode('utf-8')}")
    assert output.is_file(), f"{output} not found"
    print("OK!")
    
    
assert YOLOV4_WEIGHTS_DIR.is_dir(), f"YOLOV4_WEIGHTS_DIR={YOLOV4_WEIGHTS_DIR} not found!"
YOLOV4_COCO_WEIGHTS = YOLOV4_WEIGHTS_DIR / 'yolov4.weights'
assert YOLOV4_COCO_WEIGHTS.is_file(), f"YOLOV4_COCO_WEIGHTS={YOLOV4_COCO_WEIGHTS} not found!"
print(f"YOLOV4_COCO_WEIGHTS='{YOLOV4_COCO_WEIGHTS}")

# CONVERSION

CURR_DIR = os.getcwd()
os.chdir(YOLOV4_SUB_MOD_ROOT)
try:
    
    if not YOLOV4_COCO_TFLITE_WEIGHTS.is_file():
        convert(output=YOLOV4_COCO_TFLITE_WEIGHTS)
    assert YOLOV4_COCO_TFLITE_WEIGHTS.is_file(), f"{output} not found"
    
    if not YOLOV4_COCO_TFLITE_FP16_WEIGHTS.is_file():
        convert(output=YOLOV4_COCO_TFLITE_FP16_WEIGHTS, quant_mode='--quantize_mode float16')
    assert YOLOV4_COCO_TFLITE_FP16_WEIGHTS.is_file(), f"{output} not found"
        
    if not YOLOV4_COCO_TFLITE_UINT8_WEIGHTS.is_file():
        try:
          convert(output=YOLOV4_COCO_TFLITE_UINT8_WEIGHTS, quant_mode='--quantize_mode full_int8')
          assert YOLOV4_COCO_TFLITE_UINT8_WEIGHTS.is_file(), f"{output} not found"
        except:
          warnings.warn("conversion to UINT8 fails to be investigated")
    
except:
    raise
finally:
    os.chdir(CURR_DIR)

In [ ]:
print("The End")